In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scrapping_with_selenium as sc  # scrapping_with_selenium: past project which scrapes data from wuzzuf(check it in my past repo)

In [ ]:
# the function which scrapes the data for the firt 150
df = sc.scraper("Data", 150, ["Data Engineer","Analyst", "Data Scientist", "Bi", "Business Intelligence","Data Entry"])

In [ ]:
df.to_csv("D:\Projects\Data Roles Market\df.csv")

In [3]:
# read the csv file 
table = pd.read_csv("D:\Projects\Data Roles Market\df.csv")
table.drop(columns=table.columns[0], axis=1,  inplace=True) # drop first unnamed index column

In [4]:
table= table[
table["job title"].str.contains("Data") |
table["job title"].str.contains("Analyst") |
table["job title"].str.contains("BI")
]


In [5]:
# create a list of our conditions
conditions = [
table["job title"].str.contains("entry"),
table["job title"].str.contains("Entry"),
table["job title"].str.contains("Analyst"),
table["job title"].str.contains("BI"),
table["job title"].str.contains("Engineer"),
table["job title"].str.contains("Scientist")
]

# create a list of the values we want to assign for each condition
values = ['data entry', 'data entry', 'data analyst', 'bi developer','data engineer', 'data scientist']

# create a new column and use np.select to assign values to it using our lists as arguments
table['roles'] = np.select(conditions, values)

In [6]:
table[["city","state","country"]] = table['location'].str.split(',', expand=True)
 

In [7]:
table["experience"] = table["experience"].str.replace(" ", "")
table["experience"] = table["experience"].str.findall(".(.+?)Y")
table["experience"] = [''.join(map(str, l)) for l in table["experience"]]

In [8]:
# create a list of our conditions
conditions = [
table["experience"].str.contains("^0.*"),
table["experience"].str.contains("^1.*"),
table["experience"].str.contains("^[23].*"),
table["experience"].str.contains("^[45].*"),
table["experience"].str.contains("^[67].*")
]

# create a list of the values we want to assign for each condition
values = ['entry level', 'junior', 'mid level', 'senior','team lead']

# create a new column and use np.select to assign values to it using our lists as arguments
table['levels'] = np.select(conditions, values)